In [24]:
# %%writefile box.py
from copy import deepcopy

class Box:
    def __init__(self, x, y):
        self.coordinates = [(x, y), (x + 1, y), (x, y + 1), (x + 1, y + 1)]
        
        self.XY = (x, y)

        # lines
        self.TopLine = (self.coordinates[0], self.coordinates[1])
        self.LeftLine = (self.coordinates[0], self.coordinates[2])
        self.RightLine = (self.coordinates[1], self.coordinates[3])
        self.BottomLine = (self.coordinates[2], self.coordinates[3])
        # lines 
        self.lines = [self.TopLine, self.LeftLine, self.RightLine, self.BottomLine]

        # lines connection indicator 
        self._top = False
        self._left = False
        self._right = False
        self._bottom = False

        self.owner = None
        self.completed = False

        self.value = 1

    def connect(self, coordinates):
        line = coordinates
        success = False

        if line not in self.lines:
            return False
        
        if line == self.TopLine and self._top is False:
            self._top = True
            success = True
        elif line == self.LeftLine and self._left is False:
            self._left = True
            success = True
        elif line == self.RightLine and self._right is False:
            self._right = True
            success = True
        elif line == self.BottomLine and self._bottom is False:
            self._bottom = True
            success = True

        if self._top == True and self._bottom == True and self._left == True and self._right == True:
            self.completed = True
        
        return success

    def un_connect(self, coordinates):
        line = coordinates
        if line in self.lines:
            self.completed = False
            # self.owner = None

        if line == self.TopLine:
            self._top = False
        elif line == self.LeftLine:
            self._left = False
        elif line == self.RightLine:
            self._right = False
        elif line == self.BottomLine:
            self._bottom = False

        
    def copy(self):
        return deepcopy(self)
    
    def _repr_pretty_(self, p, cycle):
        if cycle:
            pass

        if self._top:
            p.text("*---*")
        else:
            p.text("*   *")
        p.break_()
        if self._left:
            p.text("|")
        else:
            p.text(" ")
        
        if self.completed:
            p.text(f" {self.owner} ")
        else:
            p.text("   ") 

        if self._right:
            p.text("|")

        p.break_()

        if self._bottom:
            p.text("*---*")
        else:
            p.text("*   *")



In [25]:
# %%writefile board.py
from collections import deque
from copy import deepcopy

class Board:
    display_single_box = False

    def __init__(self, m, n):
        self.player_score = 0
        self.ai_score = 0
        self.m = m
        self.n = n
        self._boxes = self._generate_boxes(m, n)
        self._open_vectors = self._generate_vectors(m, n)
        self._moves = []

    def _generate_boxes(self, rows, cols):
        """
        This function generates the boxes of the board
        """
        boxes = [[Box(x, y) for x in range(cols)] for y in range(rows)]

        return boxes
    
    def _generate_vectors(self, m, n):
        '''
        The vectors represent the available moves, or lines, which can
        be played on a game board of m rows and n columns. These are stored as tuples
        containing each coordinate and are stored in a queue. The vector queue, along
        with the list of boxes that correspond to the coordinates, are used to represent
        game state.
        Vector format: ((x1, y1), (x2, y2)).

        The vectors always point away from the origin (0, 0), so moving like (1, 0) => (0, 0)
        is not a valid move while (0, 0) => (1, 0) is a valid move
        '''
        vectors = set()
        for i in range(0, m + 1):
            for j in range(0, n):
                # Adding horizontal line vectors
                vectors.add(((j, i), (j + 1, i)))
                # Adding vertical line vectors if not in the last row
                if i < m:
                    vectors.add(((j, i), (j, i + 1)))
            # Adding the vertical line for the last column in the current row
            if i < m:
                vectors.add(((n, i), (n, i + 1)))
        return vectors

    def move(self, coordinates, player_move: bool = False):
        player = "P" if player_move == True else "A"
        
        if coordinates not in self._open_vectors:
            return False
        
        self._open_vectors.remove(coordinates)
        self._moves.append(coordinates)
        closed = self._checkboxes(coordinates, player)
            # print("board: moved")
        return closed
    
    def undo_last_move(self):
        if self._moves:
            move = self._moves.pop()
            self._open_vectors.add(move)
            for i in range(0, self.m):
                for j in range(0, self.n):
                    box = self._boxes[i][j]
                    if move in box.lines:
                        if box.completed is True:
                            if box.owner == "P":
                                self.player_score -= 1
                            else:
                                self.ai_score -= 1
                            box.owner = None
                        box.un_connect(move)                
        
    def has_moves(self):
        # return len(self._open_vectors) != 0
        return bool(self._open_vectors)

    def get_available_moves(self):
        return self._open_vectors
        
    def _checkboxes(self, coordinates, player: str):
        closed = None
        for i in range(self.m):
            for j in range(self.n):
                box = self._boxes[i][j]
                if coordinates in box.lines:
                    box.connect(coordinates)
                if box.completed == True and box.owner == None:
                    box.owner = player
                    if player == "P":
                        self.player_score += 1
                    else:
                        self.ai_score += 1
                    closed = True
        return closed

    
    def copy(self):
        return deepcopy(self)

    def display_board(self):
        # Display player scores
        print(f"Player 1: {self.player_score}")
        print(f"Player AI: {self.ai_score}\n")

        # Display the board
        for i in range(self.m):
            # Top line of each row
            top_line = "   "  # Start with some spacing for alignment
            middle_line = "   "  # Line below to display vertical lines and boxes
            
            for j in range(self.n):
                # Dot
                top_line += "*"

                # Horizontal line
                if ((j, i), (j + 1, i)) in self._moves:
                    top_line += "---"
                else:
                    top_line += "   "
                
                # Vertical line and box
                if ((j, i), (j, i + 1)) in self._moves:
                    middle_line += "|"
                else:
                    middle_line += " "
                
                # Box ownership
                if self._boxes[j][i].completed:
                    middle_line += f" {self._boxes[j][i].owner} "
                else:
                    middle_line += "   "
            
            # Last dot on the right end of the row
            top_line += "*"
            if ((self.n, i), (self.n, i + 1)) in self._moves:
                middle_line += "|"
            else:
                middle_line += " "
            
            # Print the top line and the middle line
            print(top_line)
            print(middle_line)

        # Bottom line of the last row
        bottom_line = "   "
        for j in range(self.n):
            bottom_line += "*"
            if ((j, self.m), (j + 1, self.m)) in self._moves:
                bottom_line += "---"
            else:
                bottom_line += "   "
        bottom_line += "*"
        
        # Print the bottom line
        print(bottom_line)
        print("")  # New line for spacing
    
    def _repr_pretty_(self, p, cycle):
        if (cycle):
            pass

        # Display player scores
        # p.text(f"    AI Score : {self.aiScore}\n")
        # p.text(f"Player Score : {self.playerScore}\n")

        if Board.display_single_box:
            self.__display_single_box(p)
        else:
            self.__display_multi_box(p)
        
    def __display_single_box(self, p):
        last_line = ""
        for i in range(self.m - 1):
            top_line = "\t*"
            middle_line = "\t"
            last_line = "\t*"

            for j in range(self.n):
                box = self._boxes[i][j]
                top_line += "---*" if box._top else "   *"
                middle_line += "|" if box._left else " "
                if box.completed:
                    middle_line += f" {box.owner} "
                else:
                    middle_line += "   "
                last_line += "---*" if box._bottom else "   *"

            
            if box and box._right:
                middle_line += "|"
            
            p.text(top_line)
            p.break_()
            p.text(middle_line)
            p.break_()

        p.text(last_line)
        p.break_()
    
    def __display_multi_box(self, p):
        for i in range(self.m):
            top_line = "\t"
            middle_line = "\t"
            bottom_line = "\t"

            for j in range(self.n):
                box = self._boxes[i][j]
                top_line += "*---*" if box._top else "*   *"
                middle_line += "|" if box._left else " "
                if box.completed:
                    middle_line += f" {box.owner} "
                else:
                    middle_line += "   "
                middle_line += "|" if box._right else " "
                bottom_line += "*---*" if box._bottom else "*   *"
            
            p.text(top_line)
            p.break_()
            p.text(middle_line)
            p.break_()
            p.text(bottom_line)
            p.break_()
            


In [26]:
Board.display_single_box = False

In [27]:
Board(3, 3)

	*   **   **   *
	               
	*   **   **   *
	*   **   **   *
	               
	*   **   **   *
	*   **   **   *
	               
	*   **   **   *


In [28]:
# from board import Board

board = Board(2, 2)
board

	*   **   *
	          
	*   **   *
	*   **   *
	          
	*   **   *


In [29]:
board.move(((0, 1), (1, 1)))
board

	*   **   *
	          
	*---**   *
	*---**   *
	          
	*   **   *


In [30]:
board._boxes[0][0]

*   *
    
*---*

In [46]:
# %%writefile gamemanager.py
from copy import deepcopy
from typing import Optional
import random

class GameManager:
    def __init__(self, m, n, level, mode = "minimax"):
        self._level = level
        self._board = Board(m, n)

        if mode == "alphabeta":
            self._mode = self.alpha_beta
        else:
            self._mode = self.mini_max
    
    def _repr_pretty_(self, p, cycle):
        if (cycle):
            pass

        # Display player scores
        p.text(f"    AI Score : {self._board.ai_score}\n")
        p.text(f"Player Score : {self._board.player_score}\n")
        self._board._repr_pretty_(p, cycle)

    def get_victor(self):
        print("The game ended")
        print(f"Score: Player={self._board.player_score}, AI={self._board.ai_score}")
        
        if self._board.player_score > self._board.ai_score:
            victor = "player"
        elif self._board.player_score < self._board.ai_score:
            victor = "ai"
        else:
            victor = "draw"

        return victor

    def ai_start(self):
        pass

    # def get_move(self, origin, dest) -> tuple[Optional[tuple[tuple[int, int], tuple[int, int]]], Optional[str]]:
    #     # if origin[0] > dest[0] or origin[1] > dest[1]:
    #     #     origin, dest = dest, origin
    #     coordinates = (origin, dest)
    #     # print("entered move=", coordinates)

    #     valid_move = self._board.move(coordinates, player_move = True)

    #     # self._board.move(coordinates, player_move = True)

    #     if valid_move is None:
    #         board = deepcopy(self._board)
    #         next = self._mode(board, self._level, True)
    #         # print("next move", next[1])
    #         self._board.move(next[1])
    #         if not self._board.has_moves():
    #             return (next[1], self.get_victor())
            
    #         return (next[1], None)
    #     elif valid_move:
    #         if not self._board.has_moves():
    #             return (None, self.get_victor())
            
    #         print("player got point, plays again")
    #     else:
    #         print("Invalid player move")
        
    #     return (None, None)
    
    def get_move(self, origin, dest) -> tuple[Optional[tuple[tuple[int, int], tuple[int, int]]], Optional[str]]:
        coordinates = (origin, dest)

        # Movimiento del jugador
        player_completed_box = self._board.move(coordinates, player_move=True)

        if player_completed_box is None:
            print("Invalid player move")
            return (None, None)
        
        if player_completed_box:
            print("Player got point, plays again")
            if not self._board.has_moves():
                return (None, self.get_victor())
            return (coordinates, None)  # Devuelve el movimiento y sigue en turno

        # Si el jugador no completó un cuadro, es el turno de la IA
        if not self._board.has_moves():
            return (None, self.get_victor())

        # Movimiento de la IA
        board_copy = deepcopy(self._board)
        _, ai_move = self._mode(board_copy, self._level, max_min=True)
        ai_completed_box = self._board.move(ai_move, player_move=False)
        
        print("AI move:", ai_move)
        if not self._board.has_moves():
            return (ai_move, self.get_victor())

        if ai_completed_box:
            print("AI got point, plays again")
            if not self._board.has_moves():
                return (None, self.get_victor())
            return (ai_move, None)  # Devuelve el movimiento y sigue en turno

        return (ai_move, None)  # Cambia el turno de nuevo al jugador


    def mini_max(self, state: Board, ply: int, max_min: bool) -> tuple[float, Board]:
        if ply == 0 or len(state._open_vectors) == 0:
            h = self.evaluate(state)
            return (h, None)
        
        best_move = None
        
        available_moves = list(state.get_available_moves())
        # ai turn
        if max_min is True:
            max_val = float('-inf')
            for move in available_moves:
                # new_state = deepcopy(state)
                # new_state.move(move, player_move = False)
                state.move(move, player_move = False)
                eval, _ = self.mini_max(state, ply - 1, False)
                if eval > max_val:
                    max_val = eval
                    best_move = move
                state.undo_last_move()
            
            return (max_val, best_move)
        
        # player turn
        else:
            min_val = float('inf')
            for move in available_moves:
                # new_state = deepcopy(state)
                # new_state.move(move, player_move = True)
                state.move(move, player_move = True)
                eval, _ = self.mini_max(state, ply - 1, True)
                if eval < min_val:
                    min_val = eval
                    best_move = move
                state.undo_last_move()
            
            return (min_val, best_move)
    
    def alpha_beta(self, state: Board, ply: int, is_max: bool, alpha = float('-inf'), beta = float("inf")) -> tuple[float, Board]:
        if ply == 0 or not state.has_moves():
            h = self.evaluate(state)
            return (h, None)

        best_move = None

        available_moves = list(state.get_available_moves())
        if is_max:
            max_val = float("-inf")
            for move in available_moves:
                new_state = deepcopy(state)
                new_state.move(move, player_move = False)
                eval, _ = self.alpha_beta(new_state, ply - 1, False, alpha, beta)
                if eval > max_val:
                    max_val = eval
                    best_move = move
                alpha = max(alpha, max_val)

                state.undo_last_move()
                if beta <= alpha:
                    break

            return (max_val, best_move)
        else:
            min_val = float('inf')
            for move in available_moves:
                new_state = deepcopy(state)
                new_state.move(move, player_move = True)
                eval, _ = self.alpha_beta(new_state, ply - 1, True, alpha, beta)
                if eval < min_val:
                    min_val = eval
                    best_move = move
                
                beta = min(beta, min_val)

                state.undo_last_move()
                if beta <= alpha:
                    break

            return (min_val, best_move)                

    def evaluate(self, state: Board):
        return random.randint(0, 100)
    


In [32]:
game_min = GameManager(10, 10, 3)
print(game_min._mode)
game_min

<bound method GameManager.mini_max of <__main__.GameManager object at 0x000002C5A66CA350>>


    AI Score : 0
Player Score : 0
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   

In [33]:
game_alpha = GameManager(10, 10, 10, mode="alphabeta")
print(game_alpha._mode)
game_alpha

<bound method GameManager.alpha_beta of <__main__.GameManager object at 0x000002C5A64F8E50>>


    AI Score : 0
Player Score : 0
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   

In [11]:
# move = game_min.get_move((0, 0), (0, 1))
# print(move)
# game_min

(((1, 0), (2, 0)), None)


    AI Score : 0
Player Score : 0
	*   **---**   **   **   **   **   **   **   **   *
	|                                                 
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   **   **   **   **   *
	                                                  
	*   **   **   **   **   **   **   **   **   **   *
	*   **   **   **   **   **   

In [12]:
# move = game_alpha.get_move((0, 0), (0, 1))
# print(move)
# game_alpha

KeyboardInterrupt: 

In [47]:

import random

class Tester:
    def __init__(self, board_size=(4, 4), level=3, mode="minimax"):
        self.board_size = board_size
        self.level = level
        self.mode = mode
        self.game_manager = GameManager(board_size[0], board_size[1], level, mode)
    
    def simulate_game(self, player_starts=True):
        print("\nStarting a new game...")
        self.game_manager = GameManager(self.board_size[0], self.board_size[1], self.level, self.mode)
        current_turn = "player" if player_starts else "ai"

        while self.game_manager._board.has_moves():
            print(f"\nCurrent Turn: {current_turn.upper()}")
            
            if current_turn == "player":
                # Movimiento aleatorio del jugador (puedes cambiar esto por una entrada manual si lo deseas)
                move = random.choice(list(self.game_manager._board.get_available_moves()))
                _, result = self.game_manager.get_move(*move)
                current_turn = "ai"
            else:
                # Movimiento de la IA
                move, result = self.game_manager.get_move(*random.choice(list(self.game_manager._board.get_available_moves())))
                current_turn = "player"
            
            # Imprimir el tablero y la evaluación heurística después de cada movimiento
            self.print_board_and_heuristic()
            
            if result:
                break
        
        if result:
            print("\nThe game ended")
            print(f"Final Score: Player={self.game_manager._board.player_score}, AI={self.game_manager._board.ai_score}")
            return self.game_manager.get_victor()
        return None

    def print_board_and_heuristic(self):
        # Imprime el tablero actual
        self.game_manager._board.display_board()
        
        # Calcula e imprime la evaluación heurística del estado actual
        heuristic_value = self.game_manager.evaluate(self.game_manager._board)
        print(f"Heuristic Value: {heuristic_value}\n")




In [48]:
# Inicializar el tester
tester = Tester(board_size=(4, 4), level=3, mode="minimax")

# Simular un juego con impresión de cada movimiento
victor = tester.simulate_game(player_starts=True)
print(f"\nGame Over! The winner is: {victor}")


Starting a new game...

Current Turn: PLAYER
Invalid player move
Player 1: 0
Player AI: 0

   *   *   *   *   *
                    
   *   *   *   *   *
       |            
   *   *   *   *   *
                    
   *   *   *   *   *
                    
   *   *   *   *   *

Heuristic Value: 63


Current Turn: AI
Invalid player move
Player 1: 0
Player AI: 0

   *   *   *   *   *
                    
   *   *   *   *   *
       |            
   *   *   *   *---*
                    
   *   *   *   *   *
                    
   *   *   *   *   *

Heuristic Value: 46


Current Turn: PLAYER
Invalid player move
Player 1: 0
Player AI: 0

   *---*   *   *   *
                    
   *   *   *   *   *
       |            
   *   *   *   *---*
                    
   *   *   *   *   *
                    
   *   *   *   *   *

Heuristic Value: 3


Current Turn: AI
Invalid player move
Player 1: 0
Player AI: 0

   *---*   *   *   *
                    
   *   *   *---*   *
       |         